### Question 1

Dans le cadre de la régression, on peux choisir comme fonction de perte l'erreur quadratique moyenne qui équivaut à une réduction de la variance et qui minimise la perte L2 en effectuant la moyenne des noeux terminaux.

On peut aussi choisir de minimiser la perte L1 en utilisant cette fois ci la médiane des noeux terminaux.

Il existe aussi d'autres critères comme l'erreur quadratique moyenne de Friedman, ou le critère de Poisson.

### Question 2

On simule à l'aide de rand_checkers des échantillons de taille $n=456$.
Ces données serviront à construire deux arbres, l'un utilisant le critère de Gini et l'autre utilisant l'entropie.
On affiche ensuite les courbes donnant le pourcentage d'erreur en fonction de la profondeur de l'arbre.


In [1]:
#| echo: false

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz
from matplotlib import rc

from sklearn import tree, datasets, model_selection
from tp_arbres_source import (rand_checkers, frontiere)


rc('font', **{'family': 'sans-serif', 'sans-serif': ['Computer Modern Roman']})
params = {'axes.labelsize': 6,
          'font.size': 12,
          'legend.fontsize': 12,
          'text.usetex': False,
          'figure.figsize': (10, 12)}
plt.rcParams.update(params)

sns.set_context("poster")
sns.set_palette("colorblind")
sns.set_style("white")
_ = sns.axes_style()

In [2]:
#| echo: false
# Construction des Arbres
np.random.seed(42)
dt_entropy = tree.DecisionTreeClassifier(criterion="entropy")
dt_gini = tree.DecisionTreeClassifier(criterion="gini")

# 
n1 = 114
n2 = 114
n3 = 114
n4 = 114
sigma = 0.1

data = rand_checkers(n1, n2, n3, n4, sigma)
n_samples = len(data)
X = data[:, : -1]
Y = data[:, -1].astype(int) # careful with the type (cast to int)

# Fit
dt_gini.fit(X, Y)
dt_entropy.fit(X, Y)

print("Gini criterion")
print(dt_gini.score(X, Y))

print("Entropy criterion")
print(dt_entropy.score(X, Y))

Gini criterion
1.0
Entropy criterion
1.0


In [3]:
#| echo: false

dmax = 12
scores_entropy = np.zeros(dmax)
scores_gini = np.zeros(dmax)

for i in range(dmax):
    dt_entropy = tree.DecisionTreeClassifier(criterion="entropy", max_depth=i+1)
    dt_entropy.fit(X, Y)
    scores_entropy[i] = dt_entropy.score(X, Y)

    dt_gini = tree.DecisionTreeClassifier(criterion="gini", max_depth=i+1)
    dt_gini.fit(X, Y)
    scores_gini[i] = dt_gini.score(X,Y)

plt.figure()
plt.plot(scores_entropy, label='Entropy criterion')
plt.plot(scores_gini, label='Gini criterion')
plt.legend(loc='upper left')
plt.xlabel('Max depth')
plt.ylabel('Accuracy Score')
plt.draw()

<Figure size 3000x3600 with 1 Axes>

In [4]:
#| echo: false
#| output: false
print("Scores with entropy criterion: ", scores_entropy)
print("Scores with Gini criterion: ", scores_gini)

Scores with entropy criterion:  [0.25892857 0.27455357 0.28348214 0.296875   0.32589286 0.37946429
 0.55580357 0.63169643 0.859375   0.92410714 0.94866071 0.98214286]
Scores with Gini criterion:  [0.25892857 0.27455357 0.29464286 0.33258929 0.39955357 0.52678571
 0.58258929 0.74553571 0.84598214 0.95535714 0.95758929 0.984375  ]


Sans grande surprise on trouve que la profondeur qui minimise le pourcentage d'erreur (ou maximise le score) est tout simplement la profondeur maximale de notre boucle, ici sa valeur est de 12.

### Question 3

En utilisant la profondeur qui minimise le pourecntage d'erreur (ou maximise le score) on trouve la classification suivante:

In [5]:
dt_entropy.max_depth = 12

plt.figure()
frontiere(lambda x: dt_entropy.predict(x.reshape((1, -1))), X, Y, step=100)
plt.title("Best frontier with entropy criterion")
plt.draw()
print("Best scores with entropy criterion: ", dt_entropy.score(X, Y))

Best scores with entropy criterion:  0.9821428571428571


<Figure size 3000x3600 with 2 Axes>

Le score étant très proche de 1, le modèle est 'parfait', on est dans le cas de l'overfitting, ce modèle n'est donc pas adapté.
Comme on peut le voir dans la figure ci-dessus, certaines partitions sont de trop et se généraliseront très mal sur de nouvelles données.

### Question 4

In [6]:
#| echo: false
#| output: false

tree.plot_tree(dt_entropy)
data = tree.export_graphviz(dt_entropy, filled=True, rounded=True)
graph = graphviz.Source(data)
graph.render('./binary_tree_entropy', format='pdf')

'binary_tree_entropy.pdf'

<Figure size 3000x3600 with 1 Axes>

![Arbre de décision](./binary_tree_entropy.pdf)

Voici l'arbre de décision que l'on obtient, chaque noeud correspond à une condition Vrai/Faux qui nous donnera un partionnement de l'ensemble des données. Si la valeur du booléen est vrai on passera au noeud suivant correspondant, sinon on passera au noeud correspond à Faux.
On continue ce processus jusqu'à atteindre la profondeur de l'arbre.

### Question 5

On va maintenant utiliser les arbres précedemment trouvés pour calculer le score sur un nouvel échantillon test avec $n=160$

In [7]:
data_test = rand_checkers(40, 40, 40, 40, sigma)
n_test_samples = len(data_test)
X_test = data_test[:, : -1]
Y_test = data_test[:, -1].astype(int)

In [8]:
#| echo: false

dmax = 20

scores_entropy_test = np.zeros(dmax)
scores_gini_test = np.zeros(dmax)

for i in range(dmax):
    dt_entropy = tree.DecisionTreeClassifier(criterion='entropy', max_depth=i+1)
    dt_entropy.fit(X,Y)
    scores_entropy_test[i] = dt_entropy.score(X_test, Y_test)

    dt_gini = tree.DecisionTreeClassifier(criterion='gini', max_depth=i+1)
    dt_gini.fit(X,Y)
    scores_gini_test[i] = dt_gini.score(X_test,Y_test)

plt.figure()
plt.plot(scores_entropy_test, label='Entropy criterion')
plt.plot(scores_gini_test, label='Gini criterion')
plt.legend(loc='upper left')
plt.xlabel('Max depth')
plt.ylabel('Error rate')
plt.draw()


<Figure size 3000x3600 with 1 Axes>

In [9]:
#| echo: false
#| output: false
print("Scores with entropy criterion: ", scores_entropy_test)
print("Scores with Gini criterion: ", scores_gini_test)

Scores with entropy criterion:  [0.25    0.25    0.25625 0.2625  0.3     0.325   0.5125  0.55    0.78125
 0.84375 0.8375  0.80625 0.81875 0.8375  0.83125 0.83125 0.8125  0.825
 0.8375  0.825  ]
Scores with Gini criterion:  [0.25    0.25625 0.26875 0.30625 0.35625 0.45625 0.51875 0.68125 0.76875
 0.86875 0.84375 0.84375 0.8625  0.85    0.85    0.84375 0.85    0.85
 0.85    0.84375]


On trouve ici que la profondeur qui maximise le score est de 10, pour le critère de Gini ou l'entropie.
On a donc trouvé un arbre de profondeur plus faible que précédemment, réduisant ainsi le risque d'overfitting.

## Dataset DIGITS


Dans la suite du TP on s'intéresse au jeu de données DIGITS qui contient des images 8x8 de chiffres.
On sépare les données en train/test split de 0.8/0.2.

### Question 6

In [10]:
#| echo: false

digits = datasets.load_digits()
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(digits.data, digits.target, train_size=0.8, test_size=0.2)

In [11]:
#| echo: false
#| output: false

dt_entropy = tree.DecisionTreeClassifier(criterion='entropy')
dt_gini = tree.DecisionTreeClassifier(criterion='gini')
dt_entropy.fit(X_train, Y_train)
dt_gini.fit(X_train, Y_train)

DecisionTreeClassifier()

In [12]:
#| echo: false

dmax=15

scores_entropy_test = np.zeros(dmax)
scores_gini_test = np.zeros(dmax)
scores_entropy_train = np.zeros(dmax)
scores_gini_train = np.zeros(dmax)


for i in range(dmax):
    dt_entropy = tree.DecisionTreeClassifier(criterion='entropy', max_depth=i+1)
    dt_entropy.fit(X_train,Y_train)
    scores_entropy_test[i] = dt_entropy.score(X_test, Y_test)
    scores_entropy_train[i] = dt_entropy.score(X_train, Y_train)

    dt_gini = tree.DecisionTreeClassifier(criterion='gini', max_depth=i+1)
    dt_gini.fit(X_train,Y_train)
    scores_gini_test[i] = dt_gini.score(X_test,Y_test)
    scores_gini_train[i] = dt_gini.score(X_train,Y_train)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 6))

# Training Scores
axes[0].plot(scores_entropy_train, label='Entropy criterion', marker='o')
axes[0].plot(scores_gini_train, label='Gini criterion', marker='x')
axes[0].legend(loc='lower right')
axes[0].set_title('Score on training set')
axes[0].set_xlabel('Max depth')
axes[0].set_ylabel('Accuracy Score')
axes[0].grid(True, which='both', linestyle='--', linewidth=0.5)

# Test Scores
axes[1].plot(scores_entropy_test, label='Entropy criterion', marker='o')
axes[1].plot(scores_gini_test, label='Gini criterion', marker='x')
axes[1].legend(loc='lower right')
axes[1].set_title('Score on test set')
axes[1].set_xlabel('Max depth')
axes[1].set_ylabel('Accuracy Score')
axes[1].grid(True, which='both', linestyle='--', linewidth=0.5)

plt.tight_layout()
plt.show()


<Figure size 4200x1800 with 2 Axes>

Comme dans le cas précédent on obtient les courbes de score en fonction de la profondeur de l'arbre.
Le score sur l'échantillon test semble atteindre un plateau lorsque la profondeur dépasse 7, on préfèrera alors le modèle le plus parsimonieux.

In [13]:
#| echo: false
#| output: false
print("Scores with entropy criterion: ", scores_entropy)
print("Scores with Gini criterion: ", scores_gini)

Scores with entropy criterion:  [0.25892857 0.27455357 0.28348214 0.296875   0.32589286 0.37946429
 0.55580357 0.63169643 0.859375   0.92410714 0.94866071 0.98214286]
Scores with Gini criterion:  [0.25892857 0.27455357 0.29464286 0.33258929 0.39955357 0.52678571
 0.58258929 0.74553571 0.84598214 0.95535714 0.95758929 0.984375  ]


## Sélection de modèle

### Question 7

On utilise maintenant la validation croisée pour effectuer le choix du paramètre de profondeur.

In [14]:
#| echo: false

X, y = datasets.load_digits(return_X_y=True)
scores = model_selection.cross_val_score(dt_entropy, X, y, cv=10)
print("%0.3f accuracy with a standard deviation of %0.3f" % (scores.mean(), scores.std()))

0.819 accuracy with a standard deviation of 0.055


In [15]:
#| echo: false
#| output: false

dmax = 20
X = digits.data
y = digits.target
score_entropy = np.zeros(dmax)
score_gini = np.zeros(dmax)

for i in range(dmax):
    dt_entropy = tree.DecisionTreeClassifier(criterion='entropy', max_depth=i+1)
    dt_gini = tree.DecisionTreeClassifier(criterion='gini', max_depth=i+1)

    scores_entropy = model_selection.cross_val_score(dt_entropy, X, y, cv=10)
    scores_gini = model_selection.cross_val_score(dt_gini, X, y, cv=10)
    
    score_entropy[i] = np.mean(scores_entropy)
    score_gini[i] = np.mean(scores_gini)

In [16]:
#| echo: false

plt.figure()
plt.plot(score_entropy, label='Entropy criterion')
plt.plot(score_gini, label='Gini criterion')
plt.legend(loc='lower right')
plt.xlabel('Max depth')
plt.ylabel("Accuracy Score")

max_depth_entropy = np.argmax(score_entropy) + 1
max_depth_gini = np.argmax(score_gini) + 1

print(f"The maximum cross-validation score for entropy is {score_entropy[max_depth_entropy-1]:.4f} at depth {max_depth_entropy}.")
print(f"The maximum cross-validation score for gini is {score_gini[max_depth_gini-1]:.4f} at depth {max_depth_gini}.")


The maximum cross-validation score for entropy is 0.8225 at depth 9.
The maximum cross-validation score for gini is 0.8358 at depth 12.


<Figure size 3000x3600 with 1 Axes>

### Question 8

Dans cette question on affiche la courbe d'apprentissage qui mesure l'effet du score en fonction du nombre de données durant la période d'apprentissage du modèle.

In [17]:
#| echo: false

dt_entropy = tree.DecisionTreeClassifier(criterion='entropy', max_depth=10)

fig, ax = plt.subplots(figsize=(10, 6))

common_params = {
    "X": X,
    "y": y,
    "train_sizes": np.linspace(0.1, 1.0, 10),
    "cv": model_selection.ShuffleSplit(n_splits=50, test_size=0.2, random_state=0),
    "n_jobs": 4,
    "line_kw": {"marker": "o"},
    "std_display_style": "fill_between",
    "score_name": "Accuracy",
    "score_type": "both",
}

model_selection.LearningCurveDisplay.from_estimator(dt_entropy, **common_params, ax=ax)
handles, _ = ax.get_legend_handles_labels()
ax.legend(handles[:2], ["Training Score", "Test Score"])
ax.set_title(f"Learning Curve for {dt_entropy.__class__.__name__}")

plt.show()


<Figure size 3000x1800 with 1 Axes>

Dans notre cas, le score d'apprentissage reste relativement élevé peu importe la taille de l'échantillon d'apprentissage. Cependant, le score sur l'échantillon de test augmente en fonction de la taille de l'échantillon d'apprentissage jusqu'à un certain plateau. L'ajout de nouvelles données aura un effet de moins en moins prononcé.